# Chat Completion API

In [ ]:
!pip install openai

## 환경설정
1. openai API KEY 발급
2. google colab - secret - `OPENAI_API_KEY` 등록

In [2]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [3]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [4]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고, 많은 도움을 주는 챗봇이야~"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "나는 차은우야"
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=0.1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [6]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕하세요, 차은우님! 만나서 반갑습니다. 어떤 이야기를 나누고 싶으신가요?'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context 문맥
3. Input Data/Example 입력데이터/예시
4. Output Indicator 출력 지시사항

## 기사 제목 교정
- 기자들이 송고한 기사에서 제목을 추출하고, 표현 조정
- 프랑스AFP 속보시스템에서 도입되어 사용

In [9]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요.'
title_before = '졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환'

# 역할/페르소나/지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조 등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고 의미를 적절히 유지하도록 제목을 교정하세요.

### Steps ###
1. 기사 제목을 읽고 주요내용을 이해하세요.
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성 등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈 수 있는지 점검하고 간결히 정리하세요.

### Output Format ###
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목: [기사의 원래 제목]
- 교정 제목: [기사의 교정된 제목]

### Examples ###
- 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

- 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
- 교정 제목: "전기차 판매량 급감, 반기는 내연차회사"

### Extra Instructions ###
- 제목이 너무 길면 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정 집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.
"""

user_message = f"""
다음 기사제목을 간결히 수정해주세요.

{title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

- 원래 제목: "테이의 FM 개꿀 라디오 방송에 주목해주세요."
- 교정 제목: "테이의 FM 라디오 방송에 주목하세요."


In [10]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조 등에 있어서 교정작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사 제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성 등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈 수 있는지 점검하고 간결히 정리하세요.

    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사의 원래 제목]
    - 교정 제목: [기사의 교정된 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
    - 교정 제목: "전기차 판매량 급감, 반기는 내연차회사"

    ### Extra Instructions ###
    - 제목이 너무 길면 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정 집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.
    """

    user_message = f"""
    다음 기사제목을 간결히 수정해주세요.

    {title_before}
    """

    response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {
          "role": "system",
          "content": [
            {
              "type": "text",
              "text": system_message
            }
          ]
        },
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": user_message
            }
          ]
        }
      ],
      response_format={
        "type": "text"
      },
      temperature=temperature,
      max_completion_tokens=2048,
      top_p=top_p,
      frequency_penalty=0,
      presence_penalty=0
    )
    return response.choices[0].message.content

title_before = '주말 미친 폭우 예상, 모두들 무사하시길'
output = correct_news_title(title_before)
print(output)

- 원래 제목: "주말 미친 폭우 예상, 모두들 무사하시길"
- 교정 제목: "주말 폭우 예상, 안전에 유의하세요"
